In [1]:
import sys
import sqlite3, csv
from datetime import datetime as dt
import datetime
sys.path.append('C:\\Python27\\Lib\\site-packages\\MySQLdb')

In [3]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [5]:
dbpath = "C:\\Users\\mtags\\Desktop\\sheep.db"

In [19]:
conn = sqlite3.connect(dbpath)
conn.execute("DROP TABLE sheep_tracker");
conn.execute(
    """CREATE TABLE sheep_tracker (
        actual_lot_size INTEGER, 
        lot_tag_read_count INTEGER,
        departure_cph TEXT,
        destination TEXT,
        read_location_cph TEXT,
        lot_date DATE,
        movement_type INTEGER,
        animal_eid INTEGER,
        flock_tag INTEGER,
        tag_count INTEGER,
        issue_cph TEXT,
        issue_date DATE,
        last_update DATE,
        unique_lot_id INTEGER)""")

conn.close()

In [8]:
conn = sqlite3.connect(dbpath)

conn.execute("delete from sheep_tracker");

cur = conn.cursor()

with open('ExampleData_2016_09_08.xlsx - Relevant_EXAMPLE.csv') as openFile:
    dr = csv.DictReader(openFile)    
    to_db = [(i['Actual lot size'], 
              i['lot_tag_read_count'],
              i['departurecph'],
              i['destination'],
              i['ReadLocationCPH'],
              dt.strptime(i['LotDate'], '%m/%d/%Y'),
              i['MovementType'],
              i['AnimalEID'],
              i['Flock_tag'],
              i['Tag_count'],
              i['issue_cph'],
              dt.strptime(i['issue_date'], '%m/%d/%Y'),
              dt.strptime(i['latest_update'], '%m/%d/%Y'),
              i['UniqueLotID']
             ) for i in dr]
    #print(to_db)
    for i in to_db:
        cur.execute("""
        INSERT INTO sheep_tracker (
        actual_lot_size,
        lot_tag_read_count,
        departure_cph,
        destination,
        read_location_cph,
        lot_date,
        movement_type,
        animal_eid,
        flock_tag,
        tag_count,
        issue_cph,
        issue_date,
        last_update,
        unique_lot_id 
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
        
        """, i)
        


conn.commit()

conn.close()
print("done")

done


In [74]:
conn = sqlite3.connect(dbpath)
conn.close()

### data columns 
actual_lot_size - the size of the lot that the animal belongs to (a group as they go to a desitination)?
lot_tag_read_count - the count of animals, within the lot, whose tag was read at the read_location_cph?
departure_cph - where the animal was before the reading location
destination_cph - the location where the animal is going next (a farm, a slaughter house, a shipping port, may leave the system)? after the read location
read_location_cph - the location where the animals tag was read for this record (often at a livestock or similar 3rd party)?
lot_date - the date the tag was read for this animal for this record?
movement_type - some detail about the departure and desitination - farm to farm, farm to slaughter, farm to port? This help identify that the animal has left the system (because it was killed at a slaughter)
animal_eid - the unique identifier for an animal?
flock_tag - a single tag of a full flock/lot - not individual sheep
tag_count - associated with flock tag, the number of animals scanned within the flock with the same flock tag
issue_cph - birth farm of animal, where the tag was issued to the animal
issue_date - date the animal got the tag for the issue_cph
last_update - ???
unique_lot_id - exactly

cph (county/perish/holding)


patient zero tracking
tracking window is to see when the disese started and likely ended
the slower the disease moves, the larger the window

network analysis
degrees of fredom


the result is all the sheep that the animal of interest was in contact with
    possibley of showing a weight of how long they were in contact
    show other animals current location (last scan)
    show on a pretty graph that is easy to understand
    
Also need to identify when a sheep disappeared from the system (due to lack of scanning)





### Research
Diffusion Model - Set of mathematical equations or formulas that attempts to estimate the spread of information (idea or rumor) or a contagious disease through a population.



In [9]:
%sql sqlite:///C:\\Users\\mtags\\Desktop\\sheep.db
#%sql delete from mydata
%sql select * from sheep_tracker

Done.


actual_lot_size,lot_tag_read_count,departure_cph,destination,read_location_cph,lot_date,movement_type,animal_eid,flock_tag,tag_count,issue_cph,issue_date,last_update,unique_lot_id
15,15,147/405/4891,181/722/473,34/946/125,2011-09-06 00:00:00,3,50467000025,0,1,54/336/9999,2010-08-17 00:00:00,2012-03-22 00:00:00,351905
15,15,147/405/4891,169/425/410,34/946/125,2011-09-06 00:00:00,3,50467000026,0,1,54/336/10000,2010-08-17 00:00:00,2012-01-19 00:00:00,351905
15,15,147/405/4891,169/425/410,34/946/125,2011-09-06 00:00:00,3,50467000027,0,1,54/336/10001,2010-08-17 00:00:00,2012-04-12 00:00:00,351905
15,15,147/405/4891,169/425/410,34/946/125,2011-09-06 00:00:00,3,50467000028,0,1,54/336/10002,2010-08-17 00:00:00,2012-02-23 00:00:00,351905
15,15,147/405/4891,169/425/410,34/946/125,2011-09-06 00:00:00,3,50467000029,0,1,54/336/10003,2010-08-17 00:00:00,2012-04-12 00:00:00,351905
15,15,147/405/4891,169/425/410,34/946/125,2011-09-06 00:00:00,3,50467000030,0,1,54/336/10004,2010-08-17 00:00:00,2012-04-19 00:00:00,351905
15,15,147/405/4891,169/425/410,34/946/125,2011-09-06 00:00:00,3,50467000031,0,1,54/336/10005,2010-08-17 00:00:00,2012-03-29 00:00:00,351905
15,15,147/405/4891,169/425/410,34/946/125,2011-09-06 00:00:00,3,50467000032,0,1,54/336/10006,2010-08-17 00:00:00,2012-04-05 00:00:00,351905
15,15,147/405/4891,169/425/410,34/946/125,2011-09-06 00:00:00,3,50467000033,0,1,54/336/10007,2010-08-17 00:00:00,2012-04-05 00:00:00,351905
15,15,147/405/4891,169/425/410,34/946/125,2011-09-06 00:00:00,3,50467000035,0,1,54/336/10008,2010-08-17 00:00:00,2012-03-22 00:00:00,351905


Below:
Enter the date for time_of_interest yyyy mm dd
This will query the data and return the records where the issue_date is between the time_of_interest and the time_of_interest + 28 days
It then randomly selects one of the returned records

In [20]:
from random import randint

time_of_interest = datetime.date(2011, 9, 1)
time_of_interest_end = time_of_interest + datetime.timedelta(days=28)

conn = sqlite3.connect(dbpath)
#
c1 = conn.execute("SELECT * FROM sheep_tracker WHERE lot_date BETWEEN ? AND ?", (time_of_interest, time_of_interest_end,))

all_rows = c1.fetchall()
rows_count = len(all_rows)
if rows_count > 0:
    random_row = all_rows[randint(0, rows_count - 1)]
    print(random_row)
    
    if random_row[5] < : 
    
    
else:
    print("no records found")



conn.close()

(15, 15, '147/405/4891', '169/425/410', '34/946/125', '2011-09-06 00:00:00', 3, 50467000028, 0, 1, '54/336/10002', '2010-08-17 00:00:00', '2012-02-23 00:00:00', 351905)


In [15]:
from datetime import datetime as dt
import datetime
newd = dt.strptime('8/14/2010', '%m/%d/%Y')
time_of_interest = datetime.date(2011, 9, 1)
print(newd)
print(time_of_interest)

2010-08-14 00:00:00
2011-09-01


In [23]:
%sql sqlite:///C:\\Users\\mtags\\Desktop\\sheep.db
#%sql delete from mydata
%sql select animal_eid, count(*) from sheep_tracker GROUP BY animal_eid

Done.


animal_eid,count(*)
50122300011,1
50122300012,1
50122300013,1
50122300018,1
50122300019,1
50122300021,1
50122300022,1
50122300023,1
50122300024,1
50122300026,1


## questions
1.  when selecting an animal within the time of interest, you want the records where the time_of_interest date range (TOI_start, TOI_end) overlaps with the animals time at a lot (lot_date, lot_date_end(need to find this based on next record) )  WHERE (lot_start < TIO_end AND lot_end >= TIO_start
2.  maybe need more data as each animal is only in the data once.  Or I can make some up.
3.  what will be the format of the outputs - fancy graphs or just data


# Chris's algorithm

1.  process raw data to show a sheepid, location, arrival date, departure date
    * this is where some data can be fudged to estimate lost data
2.  by selecting a sheep and a date range (disregarding the data outside of the range), determine which other sheep the primary sheep interacted with by matching locations and date overlaps
3.  (optional) apply a weight to how closely related the other sheep were to the primary sheep
    *  if sheep2 spent the whole date range with the primary sheep, then 1
    *  if only spent half the time then 0.5
    *  for each degree of separation the weight is further divided by 2 (* 0.5)
4.  show in fancy chart

# sheep location duration

In [35]:
%sql sqlite:///C:\\Users\\mtags\\Desktop\\sheep.db
%sql drop table sheep_location
%sql CREATE TABLE sheep_location(animal_eid INTEGER, location_cph VARCHAR(16), arrival_date DATE, departure_date DATE)


Done.
Done.


[]

In [9]:
%sql sqlite:///C:\\Users\\mtags\\Desktop\\sheep.db
%sql delete from sheep_location;
%sql INSERT INTO sheep_location VALUES ("s1", "l1", "2016-01-01", "2016-02-01");
%sql INSERT INTO sheep_location VALUES ("s1", "l2", "2016-02-01", "2016-03-01");
%sql INSERT INTO sheep_location VALUES ("s1", "l2", "2016-03-01", "2016-04-01");
%sql INSERT INTO sheep_location VALUES ("s1", "l2", "2016-04-01", "2016-05-01");

%sql INSERT INTO sheep_location VALUES ("s2", "l1", "2016-01-01", "2016-02-01");
%sql INSERT INTO sheep_location VALUES ("s2", "l1", "2016-02-01", "2016-03-01");
%sql INSERT INTO sheep_location VALUES ("s2", "l2", "2016-03-01", "2016-04-01");
%sql INSERT INTO sheep_location VALUES ("s2", "l2", "2016-04-01", "2016-05-01");

%sql INSERT INTO sheep_location VALUES ("s3", "l3", "2016-01-01", "2016-02-01");
%sql INSERT INTO sheep_location VALUES ("s3", "l2", "2016-02-01", "2016-03-01");
%sql INSERT INTO sheep_location VALUES ("s3", "l2", "2016-03-01", "2016-04-01");
%sql INSERT INTO sheep_location VALUES ("s3", "l2", "2016-04-01", "2016-05-01");

%sql INSERT INTO sheep_location VALUES ("s4", "l3", "2016-01-01", "2016-02-02");
%sql INSERT INTO sheep_location VALUES ("s4", "l3", "2016-02-01", "2016-03-01");
%sql INSERT INTO sheep_location VALUES ("s4", "l3", "2016-03-01", "2016-04-01");
%sql INSERT INTO sheep_location VALUES ("s4", "l3", "2016-04-01", "2016-05-01");

%sql INSERT INTO sheep_location VALUES ("s5", "l3", "2016-01-01", "2016-02-02");
%sql INSERT INTO sheep_location VALUES ("s5", "l2", "2016-02-01", "2016-03-01");
%sql INSERT INTO sheep_location VALUES ("s5", "l3", "2016-03-01", "2016-04-01");
%sql INSERT INTO sheep_location VALUES ("s5", "l3", "2016-04-01", "2016-05-01");

%sql INSERT INTO sheep_location VALUES ("s6", "l5", "2016-01-01", "2016-02-01");
%sql INSERT INTO sheep_location VALUES ("s6", "l5", "2016-02-01", "2016-03-01");
%sql INSERT INTO sheep_location VALUES ("s6", "l5", "2016-03-01", "2016-04-01");
%sql INSERT INTO sheep_location VALUES ("s6", "l5", "2016-04-01", "2016-05-01");

%sql INSERT INTO sheep_location VALUES ("s7", "l2", "2016-01-01", "2016-02-01");
%sql INSERT INTO sheep_location VALUES ("s7", "l4", "2016-02-01", "2016-03-01");
%sql INSERT INTO sheep_location VALUES ("s7", "l5", "2016-03-01", "2016-04-01");
%sql INSERT INTO sheep_location VALUES ("s7", "l5", "2016-04-01", "2016-05-01");

28 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [21]:
%sql sqlite:///C:\\Users\\mtags\\Desktop\\sheep.db
%sql delete from sheep_location;
%sql INSERT INTO sheep_location VALUES ("s1", "l1", "2016-01-01 12:00:00.000", "2016-02-01 00:00:00.000");
%sql INSERT INTO sheep_location VALUES ("s1", "l2", "2016-02-01 12:00:00.000", "2016-05-01 00:00:00.000");

%sql INSERT INTO sheep_location VALUES ("s2", "l2", "2016-01-01 12:00:00.000", "2016-03-01 00:00:00.000");
%sql INSERT INTO sheep_location VALUES ("s2", "l3", "2016-03-01 12:00:00.000", "2016-05-01 00:00:00.000");

%sql INSERT INTO sheep_location VALUES ("s3", "l3", "2016-01-01 12:00:00.000", "2016-04-01 00:00:00.000");
%sql INSERT INTO sheep_location VALUES ("s3", "l4", "2016-04-01 12:00:00.000", "2016-05-01 00:00:00.000");

%sql INSERT INTO sheep_location VALUES ("s4", "l4", "2016-01-01 12:00:00.000", "2016-05-01 00:00:00.000");

%sql INSERT INTO sheep_location VALUES ("s5", "l4", "2016-01-01 12:00:00.000", "2016-02-01 00:00:00.000");
%sql INSERT INTO sheep_location VALUES ("s5", "l1", "2016-02-01 12:00:00.000", "2016-05-01 00:00:00.000");



9 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [22]:
%sql sqlite:///C:\\Users\\mtags\\Desktop\\sheep.db
#%sql delete from mydata
%sql select * from sheep_location;

Done.


animal_eid,location_cph,arrival_date,departure_date
s1,l1,2016-01-01 12:00:00.000,2016-02-01 00:00:00.000
s1,l2,2016-02-01 12:00:00.000,2016-05-01 00:00:00.000
s2,l2,2016-01-01 12:00:00.000,2016-03-01 00:00:00.000
s2,l3,2016-03-01 12:00:00.000,2016-05-01 00:00:00.000
s3,l3,2016-01-01 12:00:00.000,2016-04-01 00:00:00.000
s3,l4,2016-04-01 12:00:00.000,2016-05-01 00:00:00.000
s4,l4,2016-01-01 12:00:00.000,2016-05-01 00:00:00.000
s5,l4,2016-01-01 12:00:00.000,2016-02-01 00:00:00.000
s5,l1,2016-02-01 12:00:00.000,2016-05-01 00:00:00.000


## algorithm to find affected sheep

In [12]:
def dosomething(conn, infected_sheeps):
    print("entering dosomething")
    new_rec_found = False
    infected_loc_dict = {}
    # 1. identify the infected sheep
    for infected_sheep in infected_sheeps:
        # 2. store all the locations that the infected sheep(s) was at and the date range that the location was infected
        
        results = conn.execute( \
            "SELECT location_cph, min(arrival_date) FROM sheep_location WHERE animal_eid = ? AND departure_date >= ? GROUP BY location_cph", \
                (infected_sheep, infected_sheeps[infected_sheep]))
        for row in results:
            infected_loc_dict[row[0]] = row[1];
            

    print(infected_loc_dict)    
    #infected_sheeps = {};
# 3. search all sheep for the infected location/time range
    for loc in infected_loc_dict:
        #print(loc + " " + infected_loc_dict[loc])
        resultsR2 = conn.execute(\
            "SELECT animal_eid, min(arrival_date) FROM sheep_location WHERE location_cph = ? AND departure_date >= ? GROUP BY animal_eid", \
                (loc, infected_loc_dict[loc],))
        
        #print('aaa' + str(infected_sheeps))
        for rowR2 in resultsR2:
            print('ddd' + str(rowR2))
            if rowR2[0] not in infected_sheeps or infected_sheeps[rowR2[0]] > rowR2[1]:
                #print(str(rowR2[0] not in infected_sheeps))                
                #if rowR2[0] in infected_sheeps:
                    #print('\t' + str(infected_sheeps[rowR2[0]] > rowR2[1]))
                
                infected_sheeps[rowR2[0]] = rowR2[1]                
                new_rec_found = True
    if new_rec_found:
        print(infected_sheeps)
        infected_sheeps = dosomething(conn, infected_sheeps)
            
    return infected_sheeps



connX = sqlite3.connect(dbpath)

#try:
#d = datetime.date(2016, 1, 1)
s = '2016-04-01'
print(d)
infected_sheeps = {'s1': s}

infected_sheeps_result = dosomething(connX, infected_sheeps)
print(infected_sheeps_result)
#except:
print("Unexpected error:", sys.exc_info()[0])
#else:
connX.close()


2016-01-01
entering dosomething
{'l2': '2016-03-01'}
ddd('s1', '2016-02-01')
ddd('s2', '2016-03-01')
ddd('s3', '2016-02-01')
ddd('s5', '2016-02-01')
{'s5': '2016-02-01', 's1': '2016-02-01', 's3': '2016-02-01', 's2': '2016-03-01'}
entering dosomething
{'l1': '2016-02-01', 'l3': '2016-01-01', 'l2': '2016-03-01'}
ddd('s1', '2016-01-01')
ddd('s2', '2016-01-01')
ddd('s3', '2016-01-01')
ddd('s4', '2016-01-01')
ddd('s5', '2016-01-01')
ddd('s1', '2016-02-01')
ddd('s2', '2016-03-01')
ddd('s3', '2016-02-01')
ddd('s5', '2016-02-01')
{'s5': '2016-01-01', 's1': '2016-01-01', 's3': '2016-01-01', 's2': '2016-01-01', 's4': '2016-01-01'}
entering dosomething
{'l1': '2016-01-01', 'l3': '2016-01-01', 'l2': '2016-03-01'}
ddd('s1', '2016-01-01')
ddd('s2', '2016-01-01')
ddd('s3', '2016-01-01')
ddd('s4', '2016-01-01')
ddd('s5', '2016-01-01')
ddd('s1', '2016-02-01')
ddd('s2', '2016-03-01')
ddd('s3', '2016-02-01')
ddd('s5', '2016-02-01')
{'s5': '2016-01-01', 's1': '2016-01-01', 's3': '2016-01-01', 's2': '2016-

In [19]:
%sql sqlite:///C:\\Users\\mtags\\Desktop\\sheep.db
#%sql delete from mydata
%sql CREATE TABLE exposure_incident ( \
    giver_animal_eid INTEGER, \
    location_cph VARCHAR(16), \
    location_exposure_date DATE, \
    receiver_animal_eid INTEGER, \
    animal_exposure_date DATE \
)


Done.


[]

In [17]:
%sql sqlite:///C:\\Users\\mtags\\Desktop\\sheep.db

%sql select locq.*, shpq.animal_eid as animal_eid_receiver, CASE WHEN arrival_date > locq.loc_exposure_date then arrival_date else locq.loc_exposure_date end AS animal_exposure_date from sheep_location shpq, \
    (select animal_eid as animal_eid_giver, location_cph, min(CASE WHEN arrival_date > '2016-02-01' then arrival_date else '2016-02-01' end) AS loc_exposure_date \
    from sheep_location where animal_eid = 's1' and departure_date >= '2016-02-01' GROUP BY animal_eid, location_cph) as locq \
    where shpq.location_cph = locq.location_cph and shpq.departure_date > locq.loc_exposure_date;

# %sql select *, location_cph, min(CASE WHEN arrival_date > '2016-02-01' then arrival_date else '2016-02-01' end) AS exposure_date \
#     from sheep_location where animal_eid = 's1' and departure_date >= '2016-02-01' ;


Done.


animal_eid_giver,location_cph,loc_exposure_date,animal_eid_receiver,animal_exposure_date
s1,l2,2016-02-01,s1,2016-02-01
s1,l2,2016-02-01,s2,2016-02-01
s1,l1,2016-02-01,s5,2016-02-01


# end sheep location duration

In [49]:
%sql sqlite:///C:\\Users\\mtags\\Desktop\\sheep.db

        
# %sql select * from sheep_location


%sql  select locq.giver_animal_eid, locq.location_cph, locq.location_exposure_date, shpq.animal_eid as receiver_animal_eid, CASE WHEN arrival_date > locq.location_exposure_date then arrival_date else locq.location_exposure_date end AS animal_exposure_date from sheep_location shpq, \
            (select animal_eid as giver_animal_eid, location_cph, min(CASE WHEN arrival_date > '2016-04-01 12:00:00.000' then arrival_date else '2016-04-01 12:00:00.000' end) AS location_exposure_date \
            from sheep_location where animal_eid = 's4' and departure_date >= '2016-04-01 12:00:00.000' GROUP BY animal_eid, location_cph) as locq \
            where shpq.location_cph = locq.location_cph and shpq.departure_date > locq.location_exposure_date
        
# %sql select animal_eid as giver_animal_eid, location_cph, min(CASE WHEN arrival_date > '2016-02-01 00:00:00.000' then arrival_date else '2016-02-01 00:00:00.000' end) AS location_exposure_date \
#             from sheep_location where animal_eid = 's1' and departure_date >= '2016-02-01 00:00:00.000' GROUP BY animal_eid, location_cph
    
# %sql select * from sheep_location \
#     where departure_date >= '2016-02-01 00:00:00.000'

Done.


giver_animal_eid,location_cph,location_exposure_date,receiver_animal_eid,animal_exposure_date
s4,l4,2016-04-01 12:00:00.000,s3,2016-04-01 12:00:00.000
s4,l4,2016-04-01 12:00:00.000,s4,2016-04-01 12:00:00.000
